# Creation of GDP and CPI Mimicking portfolios

## Data Prep

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
data = pd.read_csv('merged_data.csv', index_col=0)

In [3]:
data['def'] = np.log(data['corp_bond_return']/data['corp_bond_return'].shift(1)) - data['20y_return']
data['term'] = data['30y_return'] - data['rf']

In [4]:
data['defy'] = (data['BAA'] - data['AAA'])/100
data['termy'] = (data['10 year'] - data['1 year'])/100

In [26]:
# Predictors
predictors = data[['SL', 'SM', 'SH', 'BL', 'HM', 'BH',
                                   'def', 'term', 'defy', 'termy', 'rf']].copy().dropna()
portfolios = ['SL', 'SM', 'SH', 'BL', 'HM', 'BH']
for portfolio in portfolios:
    predictors[portfolio] = predictors[portfolio]/100 - predictors['rf'] 

In [27]:
predictors

,SL,SM,SH,BL,HM,BH,def,term,defy,termy,rf
1973-04-30,-0.116366,-0.059144,-0.054957,-0.065703,-0.030393,-0.018819,-0.003262,-0.001740,0.0083,-0.0010,0.005002
1973-05-31,-0.111271,-0.077172,-0.078682,-0.013939,-0.025844,-0.038340,0.008574,-0.013672,0.0077,-0.0023,0.005098
1973-06-30,-0.056169,-0.034510,-0.032328,-0.014313,-0.008784,-0.014188,0.000121,-0.007495,0.0076,-0.0080,0.005379
1973-07-31,0.176029,0.095916,0.084404,0.057668,0.016414,0.043144,0.041822,-0.048362,0.0079,-0.0144,0.006540
1973-08-31,-0.045824,-0.059812,-0.047233,-0.039251,-0.036736,-0.013048,-0.032932,0.026077,0.0085,-0.0122,0.006855
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.001011,-0.024650,-0.024405,-0.049074,-0.022420,-0.017718,0.000711,-0.055391,0.0108,-0.0035,0.001901
2022-09-30,-0.095166,-0.102378,-0.099396,-0.093165,-0.092168,-0.087833,-0.030357,-0.087084,0.0110,-0.0022,0.001929
2022-10-31,0.070206,0.101226,0.136357,0.052402,0.105318,0.147344,0.014156,-0.067951,0.0116,-0.0056,0.002327
2022-11-30,-0.007285,0.027681,0.023308,0.048466,0.051688,0.045547,0.017402,0.075075,0.0117,-0.0106,0.002856


In [38]:
# GDP & CPI
gdp = data['BBKMGDP'].rename('gdp').to_frame().iloc[:-3]
cpi = data['CPIAUCSL'].rename('cpi').to_frame().iloc[:-3]

# Transformations 
gdp['gdp_mom'] = gdp['gdp'] / 100
gdp['level'] = (1 + gdp['gdp_mom']).cumprod()
gdp['gdp_yoy'] = np.log(gdp['level'] / gdp['level'].shift(12)) # 4 quarter change = 12 month change

cpi['cpi_mom'] = np.log(cpi['cpi'] / cpi['cpi'].shift(1))
cpi['cpi_yoy'] = np.log(cpi['cpi'] / cpi['cpi'].shift(12))

cpi = cpi.shift(1)

## Can the base assets predict future GDP & CPI?

Vassalou (2003) use a bootstrap procedure to see if the base assets are jointly significant in predicted GDP above and beyond the control variables.

Specifically the bootstrap procedure is as follows:

1) The regression is estimated: 

<font size=3>
$
\%\Delta GDP_{t,t+12} = a + cB_{t-1,t} + kZ_{t-2,t-1} + e_{t,t+12}
$
</font>

Where B is a vector of returns on the 8 base assets, and Z is a vector of control variables.    

2) Draw a random sample (with replacement) of the explanatory variables and estimated residuals

3) Compute a GDP growth rate forcast assuming c=0

<font size=3>
$
\%\Delta \hat{GDP}_{t,t+12} = \hat{a} + \hat{k}Z_{t-2,t-1} + \hat{e}_{t,t+12}
$
</font>

4) Regress the null GDP growth rate from (3) on the bootstrapped sample of explanatory variables; compute the Wald Statistic

6) This is repeated 10,000 times. The actual Wald Statistic is compared against the bootstrapped distribution, and the emprical p-value is computed. 

### GDP Bootstrap

In [133]:
# Data
controls = sm.add_constant(predictors.iloc[:,-3:].shift(2).dropna())
constant = controls.iloc[:,0]
assets = predictors.iloc[1:,0:6].shift(1).dropna()
y = gdp['gdp_yoy'].loc['1973-04':].iloc[2:]

#### Only base assets

In [134]:
# Initial Model
model1 = sm.OLS(y, pd.concat([constant, assets], axis=1)).fit(cov_type='HAC', cov_kwds={'maxlags':3})

coeffs1_0 = [x + '=0' for x in assets]
wald_stat1 = model1.wald_test(','.join(coeffs1_0), scalar=True, use_f=False).statistic
p1 = model1.wald_test(','.join(coeffs1_0), scalar=True, use_f=False).pvalue
print('P-value=', f'{p1:0.5f}', 'Wald Stat= ', f'{wald_stat1:0.3f}')

P-value= 0.39646 Wald Stat=  6.244


In [137]:
sample_df1 = pd.concat([y, constant, assets], axis=1)
n = len(sample_df1)

In [138]:
# Bootstrap
boot1_wald = []
n_boots = 10000

for _ in range(n_boots):
    boot_df = sample_df1.sample(n=n, replace=True)
    
    boot_model = sm.OLS(boot_df['gdp_yoy'], boot_df.iloc[:,1:]).fit(cov_type='HAC', cov_kwds={'maxlags':3})
    
    coeffs_0 = [x + '=0' for x in boot_df.iloc[:,-6:]]
    boot1_wald.append(boot_model.wald_test(','.join(coeffs_0), scalar=True, use_f=False).statistic)

boot1_wald = np.array(boot1_wald)

In [141]:
empirical_p1 = np.sum(boot1_wald >= wald_stat1)/len(boot1_wald)
print(f'{empirical_p1: 0.5f}')

 0.98520


#### Base assets + Controls

In [142]:
# Initial Model
model2 = sm.OLS(y, pd.concat([controls, assets], axis=1)).fit(cov_type='HAC', cov_kwds={'maxlags':3})

coeffs2_0 = [x + '=0' for x in assets]
wald_stat2 = model2.wald_test(','.join(coeffs2_0), scalar=True, use_f=False).statistic
p2 = model2.wald_test(','.join(coeffs2_0), scalar=True, use_f=False).pvalue
print('P-value=', f'{p2:0.5f}', 'Wald Stat= ', f'{wald_stat2:0.3f}')

P-value= 0.36288 Wald Stat=  6.566


In [143]:
sample_df2 = pd.concat([y, controls, assets], axis=1)
n = len(sample_df2)

In [ ]:
# Bootstrap
boot2_wald = []
n_boots = 10000

for _ in range(n_boots):
    boot_df = sample_df2.sample(n=n, replace=True)
    
    boot_model = sm.OLS(boot_df['gdp_yoy'], boot_df.iloc[:,1:]).fit(cov_type='HAC', cov_kwds={'maxlags':3})
    
    coeffs_0 = [x + '=0' for x in boot_df.iloc[:,-6:]]
    boot2_wald.append(boot_model.wald_test(','.join(coeffs_0), scalar=True, use_f=False).statistic)

boot2_wald = np.array(boot2_wald)

In [ ]:
empirical_p2 = np.sum(boot2_wald >= wald_stat2)/len(boot2_wald)
print(f'{empirical_p2: 0.5f}')

### GDP
Since this is an expectation of monthly GDP, instead of "news related to future GDP growth", this factor would be more related to innovations in expectations of future GDP growth. 

In [15]:
# Model with test asset betas equal to 0
control_model = sm.OLS(gdp['gdp_yoy'].loc['1973-04':].iloc[2:], 
                      sm.add_constant(predictors.iloc[:,-3:].shift(2).dropna())) \
                       .fit(cov_type='HAC', cov_kwds={'maxlags':3})
control_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                gdp_yoy   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.262
Method:                 Least Squares   F-statistic:                     18.45
Date:                Wed, 15 Mar 2023   Prob (F-statistic):           1.87e-11
Time:                        09:49:10   Log-Likelihood:                -118.14
No. Observations:                 595   AIC:                             244.3
Df Residuals:                     591   BIC:                             261.8
Df Model:                           3                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3823      0.076      5.011      0.000       0.233       0.532
defy         -39.8441      5.437     -7.328      0.000     -50.500     -29.188
termy         10.8405      3.176      3.413      0.001       4.615      17.066
rf            56.5017     15.131      3.734      0.000      26.846      86.158
==============================================================================
Omnibus:                      318.535   Durbin-Watson:                   0.181
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3526.510
Skew:                          -2.124   Prob(JB):                         0.00
Kurtosis:                      14.145   Cond. No.                         467.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 3 lags and without small sample correction
"""

In [23]:
# Wald Test for joint significance
coeffs_0 = [x + '=0' for x in predictors.iloc[:,-3:].shift(2).dropna()]
control_model.wald_test(','.join(coeffs_0), scalar=True, use_f=False)

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=55.35752618897715, p-value=5.7601962441040665e-12, df_denom=3>

### CPI

In [139]:
# Model with test asset betas equal to 0
control_model = sm.OLS(cpi['cpi_yoy'].loc['1973-04':].iloc[1:], 
                      sm.add_constant(predictors.iloc[:,-3:].shift(1).dropna())) \
                       .fit(cov_type='HAC', cov_kwds={'maxlags':3})
control_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                cpi_yoy   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     50.68
Date:                Tue, 14 Mar 2023   Prob (F-statistic):           3.62e-29
Time:                        19:39:58   Log-Likelihood:                 1475.2
No. Observations:                 596   AIC:                            -2942.
Df Residuals:                     592   BIC:                            -2925.
Df Model:                           3                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0216      0.006      3.913      0.000       0.011       0.032
defy           0.6931      0.462      1.501      0.133      -0.212       1.598
termy         -0.6553      0.171     -3.827      0.000      -0.991      -0.320
rf             4.8097      0.723      6.653      0.000       3.393       6.227
==============================================================================
Omnibus:                       66.357   Durbin-Watson:                   0.081
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.405
Skew:                           0.854   Prob(JB):                     1.05e-19
Kurtosis:                       3.775   Cond. No.                         466.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 3 lags and without small sample correction
"""

### Systems Estimation

In [186]:
from linearmodels.system.model import SUR

In [164]:
# GDP and CPI Stacked
y = pd.merge(cpi['cpi_yoy'], gdp['gdp_yoy'], left_index=True, right_index=True).loc['1973-04':].iloc[1:]
X = sm.add_constant(predictors.iloc[:,-3:].shift(1).dropna())

In [181]:
model = SUR.multivariate_ls(y, X).fit(cov_type='kernel', kernel='bartlett',bandwidth=3)

In [182]:
print(model)

                           System OLS Estimation Summary                           
Estimator:                        OLS   Overall R-squared:                   0.2818
No. Equations.:                     2   McElroy's R-squared:                 0.4092
No. Observations:                 596   Judge's (OLS) R-squared:             0.2818
Date:                Tue, Mar 14 2023   Berndt's R-squared:                  0.6385
Time:                        20:18:30   Dhrymes's R-squared:                 0.2818
                                        Cov. Estimator:                      kernel
                                        Num. Constraints:                      None
                Equation: cpi_yoy, Dependent Variable: cpi_yoy                
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0216     0.0055     3.9126     0.0001      0.0108      0.0325
defy        